In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve

%run ../scripts/post_scraping_text_processing.py load

df loaded.
15425 rows x 5 columns


In [2]:
corpus = df['text'].copy()

years = pd.to_datetime(df['release']).dt.year
y = years.map(lambda x: 1 if x <2002 else 0)

In [ ]:
cvec = CountVectorizer()
cvec.fit(corpus)
matrix = cvec.transform(corpus)
X = matrix.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
sns.heatmap(confusion_matrix(y_test, gnb.predict(X_test)), fmt = 'g', annot = True)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, gnb.predict(X_test)).ravel()

sens = tp/(tp + fn)
spec = tn/(tn + fp)
print('Sensitivity:',sens)
print('Specificity:',spec)

Pretty shitty performance.

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
sns.heatmap(confusion_matrix(y_test, lr.predict(X_test)), fmt = 'g', annot = True)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, lr.predict(X_test)).ravel()

sens = tp/(tp + fn)
spec = tn/(tn + fp)
print('Sensitivity:',sens)
print('Specificity:',spec)

In [ ]:
np.mean(y)

In [ ]:
f1_score(y_test, lr.predict(X_test))

In [ ]:
for item in pd.Series(lr.coef_[0]).sort_values()[:100].index:
    print((cvec.get_feature_names()[item] + ':').ljust(15), round(pd.Series(lr.coef_[0]).sort_values()[item], 5))

In [ ]:
tfidf = TfidfVectorizer(stop_words=STOP_WORDS)
tfidf.fit(corpus)
matrix = tfidf.transform(corpus)
X = matrix.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
sns.heatmap(confusion_matrix(y_test, lr.predict(X_test)), fmt = 'g', annot = True)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, lr.predict(X_test)).ravel()

sens = tp/(tp + fn)
spec = tn/(tn + fp)
print('Sensitivity:',sens)
print('Specificity:',spec)

In [ ]:
f1_score(y_test, lr.predict(X_test))

In [ ]:
for item in pd.Series(lr.coef_[0]).sort_values(ascending = False)[:100].index:
    print((tfidf.get_feature_names()[item] + ':').ljust(15), round(pd.Series(lr.coef_[0])[item], 5))